# Sentiment Analysis

In [1]:
import pandas as pd

splits = {'train': 'train_df.csv', 'validation': 'val_df.csv', 'test': 'test_df.csv'}
dataframes = []
for split, filename in splits.items():
    if filename != 'val_df.csv':
      df = pd.read_csv(f'hf://datasets/Sp1786/multiclass-sentiment-analysis-dataset/{filename}')
      df.drop(['id', 'label'], axis=1, inplace=True)
      dataframes.append(df)
    else:
      test_df = pd.read_csv(f'hf://datasets/Sp1786/multiclass-sentiment-analysis-dataset/{filename}')
      test_df.drop(['id', 'label'], axis=1, inplace=True)
train_df = pd.concat(dataframes)
train_df = train_df.dropna()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
print(train_df.head())
print(test_df.head())

print(len(train_df))
print(len(test_df))

                                                text sentiment
0                    Cooking microwave pizzas, yummy  positive
1  Any plans of allowing sub tasks to show up in ...   neutral
2   I love the humor, I just reworded it. Like sa...  positive
3                       naw idk what ur talkin about   neutral
4          That sucks to hear. I hate days like that  negative
                                                text sentiment
0  Laying in bed til workkk... Oh the life. Defin...  negative
1   ooohhh imma need you to get on that asap love...  positive
2   Thanks! I love it they have a video, so you d...  positive
3     I left my ipod in the car so now its all warm.  positive
4  Great app. Only complaint is that I'd like the...  positive
36437
5205


## Train the Model

In [3]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 297.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 123.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 123.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 9.0 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=5718ce52af4fbff383e001ed5acb75feef936f143c166d1ee25c83c5b6574a54
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [4]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

model_args = ClassificationArgs()

model_args.overwrite_output_dir=True
model_args.evaluate_during_training=True
model_args.no_save = True

model_args.max_seq_length=256
model_args.use_early_stopping=True
model_args.early_stopping_delta=0.01
model_args.early_stopping_metric='eval_loss'
model_args.early_stopping_metric_minimize=True
model_args.early_stopping_patience=2
model_args.evaluate_during_training_steps=32
model_args.reprocess_input_data=True
model_args.manual_seed=4
model_args.use_multiprocessing=True
model_args.labels_list=["neutral", "positive", "negative"]
model_args.wandb_project="Sentiment-Analysis-Sweep"

In [5]:
import wandb

def train_model():
  wandb.init()
  model = ClassificationModel("roberta", "roberta-base", num_labels=3, args=model_args, use_cuda=True, sweep_config=wandb.config)
  model.train_model(train_df, eval_df=test_df)

In [6]:
wandb.login()

sweep_configuration = {
	"method": "grid",
	"metric": {"goal": "minimize", "name": "eval_loss"},
	"parameters": {
		"train_epochs": {"values": [8, 10, 12, 16]},
		"train_batch_size": {"values": [16, 32, 64]},
		"learning_rate": {"values": [1e-6, 5e-5, 1e-5, 5e-4]},
	}
}
sweep_id = input("Please enter an existing sweep id if you want to continue an existing sweep (leave blank for a new sweep): ")
sweep_id = None if sweep_id == "" else sweep_id

if sweep_id is None:
	sweep_id = wandb.sweep(sweep=sweep_configuration, project="Sentiment-Analysis-Sweep")

wandb.agent(sweep_id, function=train_model, project='Sentiment-Analysis-Sweep')

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: simonas-mickus (d3dsec) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Please enter an existing sweep id if you want to continue an existing sweep (leave blank for a new sweep): 2qk2z5y6


...

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:905: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


Training loss,█▆▁
eval_loss,█▁▁▁▂
global_step,▁▂▃▅▅▆▇█
lr,█▄▁
mcc,█▁▁▁▁
train_loss,█▁▄▃▂
Training loss,1.07015
eval_loss,1.10271
global_step,160
lr,0.00039
mcc,0


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [16]:
best_model_args = ClassificationArgs()

best_model_args.overwrite_output_dir=True
best_model_args.evaluate_during_training=True
best_model_args.no_save = True

best_model_args.max_seq_length=256
best_model_args.use_early_stopping=True
best_model_args.early_stopping_delta=0.01
best_model_args.early_stopping_metric='eval_loss'
best_model_args.early_stopping_metric_minimize=True
best_model_args.early_stopping_patience=2
best_model_args.evaluate_during_training_steps=32
best_model_args.reprocess_input_data=True
best_model_args.manual_seed=25
best_model_args.use_multiprocessing=True
best_model_args.labels_list=["neutral", "positive", "negative"]


In [17]:
eval_df = pd.read_csv('./sentiment-topic-test.tsv', sep='\t')
eval_df = eval_df.dropna()
eval_df.drop(['sentence_id', 'topic'], axis=1, inplace=True)
eval_df.columns = ['text', 'sentiment']
print(eval_df.head())

best_model_args.num_train_epochs=16
best_model_args.train_batch_size=64
best_model_args.learning_rate=5e-5

best_model = ClassificationModel("roberta", "roberta-base", num_labels=3, args=best_model_args, use_cuda=True)

                                                text sentiment
0  The stadium was alive with the roar of the cro...  positive
1  That last-minute goal had me jumping out of my...  positive
2  I couldn’t put the book down; it swept me into...  positive
3  The story had its moments, though some parts f...   neutral
4  I enjoyed the way the timelines shifted, even ...   neutral


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
best_model.train_model(pd.concat([train_df, test_df]), eval_df=eval_df)

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/83 [00:00<?, ?it/s]

Epoch:   0%|          | 0/16 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:882: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


Running Epoch 1 of 16:   0%|          | 0/651 [00:00<?, ?it/s]

...

0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:905: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


(320,
 defaultdict(list,
             {'global_step': [32, 64, 96, 128, 160, 192, 224, 256, 288, 320],
              'train_loss': [1.1049652099609375,
               1.0863189697265625,
               0.99652099609375,
               0.8107376098632812,
               0.7526912689208984,
               0.6337594985961914,
               0.6412763595581055,
               0.6083526611328125,
               0.5375757217407227,
               0.8013091087341309],
              'mcc': [0.0,
               0.0,
               np.float64(0.49346377121982676),
               np.float64(0.6963106238227914),
               np.float64(0.7755547359889252),
               np.float64(0.5860528507952449),
               np.float64(0.8492077756084468),
               np.float64(0.7606388292556648),
               np.float64(0.5833333333333334),
               np.float64(0.6032092391024974)],
              'eval_loss': [1.0996636152267456,
               1.0993380546569824,
               1.030164957

In [20]:
from sklearn.metrics import classification_report

predict, probabilities = best_model.predict(eval_df['text'].tolist())
eval_df['predicted'] = predict
print(classification_report(eval_df['sentiment'], eval_df['predicted']))

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

              precision    recall  f1-score   support

    negative       1.00      0.50      0.67         6
     neutral       0.57      0.67      0.62         6
    positive       0.75      1.00      0.86         6

    accuracy                           0.72        18
   macro avg       0.77      0.72      0.71        18
weighted avg       0.77      0.72      0.71        18



/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:2188: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
